### TODO:
1. ellenőrizni megyéket: lett évről-évre eltérés? --> ha nincs nagy eltérés, hozzuk egy oszlopra
2. adatokat aggregálni megye illetve gazdálkodási forma és ágazat kategória szerint
3. feature_importance check
4. cross-validation
5. number of trees to use: plot the training: num trees on x and error on y axis

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk

In [2]:
from sklearn.model_selection import train_test_split

target_column = 'target_reg'

train_df = pd.read_csv('./dataset/train.csv')
evaluation_df = pd.read_csv('./dataset/test.csv')


# Clean data, reduce features

In [3]:
# train_df.dropna(inplace=True)

In [4]:
MEGYE_COLUMNS = ['MEGYE_2014', 'MEGYE_2015', 'MEGYE_2016']
SZEKHELY_COLUMNS = ['SZEKHELY_IR_SZAM_2014', 'SZEKHELY_IR_SZAM_2015', 'SZEKHELY_IR_SZAM_2016']

GAZD_FORMA_FO_KAT_COLUMNS = ['GAZDALKODASI_FORMA_2014_fo_kategoria', 'GAZDALKODASI_FORMA_2015_fo_kategoria', 'GAZDALKODASI_FORMA_2016_fo_kategoria']
GAZD_FORMA_1_ALKAT_COLUMNS = ['GAZDALKODASI_FORMA_2014_1_alkategoria', 'GAZDALKODASI_FORMA_2015_1_alkategoria', 'GAZDALKODASI_FORMA_2016_1_alkategoria']
GAZD_FORMA_2_ALKAT_COLUMNS = ['GAZDALKODASI_FORMA_2014_2_alkategoria', 'GAZDALKODASI_FORMA_2015_2_alkategoria', 'GAZDALKODASI_FORMA_2016_2_alkategoria']

KKV_BESOR_FO_KAT_COLUMNS = ['KKV_BESOROLAS_2014_fo_kategoria', 'KKV_BESOROLAS_2015_fo_kategoria', 'KKV_BESOROLAS_2016_fo_kategoria']
KKV_BESOR_1_ALKAT_COLUMNS = ['KKV_BESOROLAS_2014_1_alkategoria', 'KKV_BESOROLAS_2015_1_alkategoria', 'KKV_BESOROLAS_2016_1_alkategoria']
KKV_BESOR_2_ALKAT_COLUMNS = ['KKV_BESOROLAS_2014_2_alkategoria', 'KKV_BESOROLAS_2015_2_alkategoria', 'KKV_BESOROLAS_2016_2_alkategoria']

AGAZAT_FO_KAT_COLUMNS = ['agazat_2014_fo_kategoria', 'agazat_2015_fo_kategoria', 'agazat_2016_fo_kategoria'] 
AGAZAT_1_ALKAT_COLUMNS = ['agazat_2014_1_alkategoria', 'agazat_2015_1_alkategoria', 'agazat_2016_1_alkategoria']
AGAZAT_2_ALKAT_COLUMNS = ['agazat_2014_2_alkategoria', 'agazat_2015_2_alkategoria', 'agazat_2016_2_alkategoria']
AGAZAT_3_ALKAT_COLUMNS = ['agazat_2014_3_alkategoria', 'agazat_2015_3_alkategoria', 'agazat_2016_3_alkategoria']

COLUMN_GROUPS = [
    MEGYE_COLUMNS, 
    SZEKHELY_COLUMNS,
    GAZD_FORMA_FO_KAT_COLUMNS,
    GAZD_FORMA_1_ALKAT_COLUMNS,
    GAZD_FORMA_2_ALKAT_COLUMNS,
    KKV_BESOR_FO_KAT_COLUMNS,
    KKV_BESOR_1_ALKAT_COLUMNS,
    KKV_BESOR_2_ALKAT_COLUMNS,
    AGAZAT_FO_KAT_COLUMNS,
    AGAZAT_1_ALKAT_COLUMNS,
    AGAZAT_2_ALKAT_COLUMNS,
    AGAZAT_3_ALKAT_COLUMNS,
]

In [20]:
from itertools import combinations


def check_if_difference_in_columns(df, columns):
    for pair in combinations(columns, 2):
        column_a, column_b = pair[0], pair[1]
        differing_number_of_rows = df.loc[df[column_a] != df[column_b]].shape[0]
        if differing_number_of_rows > 0:
            print(
                f'different number of rows: {differing_number_of_rows}',
                f'found in columns: {column_a}, {column_b}',
                sep=' '
            )
        else:
            print(f'no difference in columns: {column_a}, {column_b}')

    return False

def check_column_groups_for_difference(df, column_groups):
    for group in column_groups:
        check_if_difference_in_columns(df, group)




In [12]:
evaluation_df.loc[evaluation_df['KKV_BESOROLAS_2015_fo_kategoria'] != evaluation_df['KKV_BESOROLAS_2016_fo_kategoria']].shape[0]

16

In [21]:
check_column_groups_for_difference(df=train_df, column_groups=COLUMN_GROUPS)

no difference in columns: MEGYE_2014, MEGYE_2015
no difference in columns: MEGYE_2014, MEGYE_2016
no difference in columns: MEGYE_2015, MEGYE_2016
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2015
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2016
no difference in columns: SZEKHELY_IR_SZAM_2015, SZEKHELY_IR_SZAM_2016
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2015_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2015_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2015_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2015_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GA

In [22]:
check_column_groups_for_difference(df=evaluation_df, column_groups=COLUMN_GROUPS)

no difference in columns: MEGYE_2014, MEGYE_2015
no difference in columns: MEGYE_2014, MEGYE_2016
no difference in columns: MEGYE_2015, MEGYE_2016
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2015
no difference in columns: SZEKHELY_IR_SZAM_2014, SZEKHELY_IR_SZAM_2016
no difference in columns: SZEKHELY_IR_SZAM_2015, SZEKHELY_IR_SZAM_2016
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2015_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2015_fo_kategoria, GAZDALKODASI_FORMA_2016_fo_kategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2015_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2014_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GAZDALKODASI_FORMA_2015_1_alkategoria, GAZDALKODASI_FORMA_2016_1_alkategoria
no difference in columns: GA

since some of the column groups have no difference among them, we can keep one column per group and drop the rest

In [30]:
train_df_reduced_features = train_df.copy()

column_groups_to_reduce = [
    MEGYE_COLUMNS,
    SZEKHELY_COLUMNS,
    GAZD_FORMA_FO_KAT_COLUMNS,
    GAZD_FORMA_1_ALKAT_COLUMNS,
    GAZD_FORMA_2_ALKAT_COLUMNS
]

for group in column_groups_to_reduce:
    train_df_reduced_features.drop(columns=group[1:], axis='columns', inplace=True)

pd.set_option('display.max_columns', None)
train_df_reduced_features.head()

,id,target_reg,KATEGORIKUS_VALTOZO_2014,KATEGORIKUS_VALTOZO_2015,KATEGORIKUS_VALTOZO_2016,MEGYE_2014,SZEKHELY_IR_SZAM_2014,aktelh_2014,aktelh_2015,aktelh_2016,aktsajt_2014,aktsajt_2015,aktsajt_2016,alakul_ev,arbevert_2014,arbevert_2015,arbevert_2016,arbevexp_2014,arbevexp_2015,arbevexp_2016,befpue_2014,befpue_2015,befpue_2016,begyeb_2014,begyeb_2015,begyeb_2016,celtart_2014,celtart_2015,celtart_2016,eredadel_2014,eredadel_2015,eredadel_2016,eredadoz_2014,eredadoz_2015,eredadoz_2016,eredpenz_2014,eredpenz_2015,eredpenz_2016,eredtar_2014,eredtar_2015,eredtar_2016,ereduzem_2014,ereduzem_2015,ereduzem_2016,ertpapir_2014,ertpapir_2015,ertpapir_2016,hoskot01_2014,hoskot01_2015,hoskot01_2016,hoskot_2014,hoskot_2015,hoskot_2016,immat_2014,immat_2015,immat_2016,jetok_2014,jetok_2015,jetok_2016,kecs_2014,kecs_2015,kecs_2016,keszl_2014,keszl_2015,keszl_2016,kovet_2014,kovet_2015,kovet_2016,letszam_2014,letszam_2015,letszam_2016,merlfoo_2014,merlfoo_2015,merlfoo_2016,penzeszk_2014,penzeszk_2015,penzeszk_2016,ranyag_2014,ranyag_2015,ranyag_2016,regyeb_2014,regyeb_2015,regyeb_2016,rovkot01_2014,rovkot01_2015,rovkot01_2016,rovkot_2014,rovkot_2015,rovkot_2016,rszem_2014,rszem_2015,rszem_2016,sajattok_2014,sajattok_2015,sajattok_2016,targyie_2014,targyie_2015,targyie_2016,GAZDALKODASI_FORMA_2014_fo_kategoria,GAZDALKODASI_FORMA_2014_1_alkategoria,GAZDALKODASI_FORMA_2014_2_alkategoria,KKV_BESOROLAS_2014_fo_kategoria,KKV_BESOROLAS_2014_1_alkategoria,KKV_BESOROLAS_2014_2_alkategoria,KKV_BESOROLAS_2015_fo_kategoria,KKV_BESOROLAS_2015_1_alkategoria,KKV_BESOROLAS_2015_2_alkategoria,KKV_BESOROLAS_2016_fo_kategoria,KKV_BESOROLAS_2016_1_alkategoria,KKV_BESOROLAS_2016_2_alkategoria,agazat_2014_fo_kategoria,agazat_2014_1_alkategoria,agazat_2014_2_alkategoria,agazat_2014_3_alkategoria,agazat_2015_fo_kategoria,agazat_2015_1_alkategoria,agazat_2015_2_alkategoria,agazat_2015_3_alkategoria,agazat_2016_fo_kategoria,agazat_2016_1_alkategoria,agazat_2016_2_alkategoria,agazat_2016_3_alkategoria
0,1,1907229.0,0,0,0,Győr-Moson-Sopron,zipcode_1,457724.0,586098.0,643830.0,535126.0,849583.0,1234907.0,1976.0,13983828.0,12699378.0,11432950.0,1382989.0,1700751.0,1529118.0,1855172.0,1849289.0,1839898.0,15456185.0,15568368.0,19590889.0,1490863.0,2806727.0,3665800.0,939106.0,773314.0,703532.0,706406.0,433306.0,500046.0,-145397.0,35709.0,-83352.0,37983621.0,55695087.0,89228704.0,-2661421.0,-2520379.0,786884.0,0.0,0.0,0.0,952490.0,521178.0,2885261.0,48333543.0,45717629.0,45942043.0,132219.0,103981.0,84122.0,7520242.0,7520242.0,7520242.0,5547891.0,5849322.0,5686293.0,1395646.0,1567766.0,1798279.0,9012948.0,7550810.0,6199791.0,1845.0,1849.0,1845.0,121166809.0,145682514.0,170549985.0,1181661.0,3878508.0,3592889.0,14700476.0,12925752.0,13101480.0,2144006.0,2135752.0,1412180.0,9752906.0,3854551.0,5805061.0,19908755.0,27579737.0,17326862.0,10244187.0,10726882.0,11271908.0,13450028.0,13883334.0,14386576.0,107131440.0,130146063.0,156391175.0,P,P,A9,7-R,A,A,7-R,A,A,7-R,A,A,A9,9,P,A,A9,9,P,A,A9,9,P,A
1,2,0.0,0,0,0,Pest,zipcode_521,49.0,720.0,421.0,0.0,0.0,0.0,2005.0,41717.0,51525.0,60916.0,0.0,0.0,0.0,0.0,0.0,0.0,104.0,9346.0,0.0,0.0,0.0,0.0,7723.0,3361.0,56.0,6950.0,3024.0,-66.0,22.0,-110.0,-428.0,0.0,0.0,0.0,7701.0,3471.0,484.0,0.0,0.0,0.0,0.0,5026.0,3693.0,0.0,5026.0,3693.0,0.0,0.0,338.0,5497.0,5497.0,5497.0,2536.0,1891.0,3463.0,0.0,0.0,0.0,5200.0,6087.0,5327.0,7.0,6.0,6.0,32023.0,44129.0,40447.0,23968.0,27858.0,26844.0,19031.0,35572.0,45404.0,1059.0,9354.0,1059.0,1724.0,4614.0,3496.0,1724.0,5778.0,3496.0,11494.0,10583.0,10506.0,30300.0,33324.0,33258.0,2806.0,9464.0,7518.0,P,P,x,P,P,x,P,P,x,P,P,x,w-5,w-5,7-R,7-R,w-5,w-5,7-R,7-R,w-5,w-5,7-R,7-R
2,3,0.0,0,0,0,Fejér,zipcode_30,80.0,538.0,680.0,0.0,0.0,0.0,2003.0,181193.0,146301.0,156244.0,0.0,0.0,0.0,0.0,0.0,0.0,7538.0,0.0,0.0,0.0,0.0,0.0,119017.0,67182.0,76190.0,107115.0,60323.0,68571.0,1116.0,386.0,106.0,0.0,0.0,0.0,117806.0,67126.0,76085.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3301.0,3301.0,3301.0,4112.0,6862.0,5454.0,0.0,0

In [31]:


train_df_reduced_features = train_df.copy()

train_df_reduced_features.drop(['MEGYE_2015', 'MEGYE_2016'], axis='columns', inplace=True)
train_df_reduced_features.rename(columns={'MEGYE_2014': 'megye'}, inplace=True)

In [32]:
print(train_df_reduced_features.loc[train_df_reduced_features['SZEKHELY_IR_SZAM_2014'] != train_df_reduced_features['SZEKHELY_IR_SZAM_2015']].shape)
print(train_df_reduced_features.loc[train_df_reduced_features['SZEKHELY_IR_SZAM_2015'] != train_df_reduced_features['SZEKHELY_IR_SZAM_2016']].shape)
print(train_df_reduced_features.loc[train_df_reduced_features['SZEKHELY_IR_SZAM_2014'] != train_df_reduced_features['SZEKHELY_IR_SZAM_2016']].shape)


(0, 130)
(0, 130)
(0, 130)


szekhely columns don't change either, we can keep one

In [33]:
train_df_reduced_features.drop(['SZEKHELY_IR_SZAM_2014', 'SZEKHELY_IR_SZAM_2015'], axis='columns', inplace=True)
train_df_reduced_features.rename(columns={'SZEKHELY_IR_SZAM_2016': 'iranyito_szam'}, inplace=True)

### target_reg min miért negatív? létezik negatív export? + plottal meg lehetne vizsgálni

In [ ]:
print(train_df.describe())

In [ ]:
from sklearn.preprocessing import OneHotEncoder
from utils import columns_to_onehotencode

encoder = OneHotEncoder(handle_unknown='ignore')

encoder_df = pd.DataFrame(encoder.fit_transform(train_df[[*columns_to_onehotencode]]).toarray())
train_df.reset_index(inplace=True)
encoder_df.reset_index(inplace=True)
train_df = pd.concat([train_df, encoder_df], axis='columns')
print(train_df.shape)
print(train_df.isna().sum().sum())

# Train

In [ ]:
target_df = train_df[target_column]
train_df.drop([target_column, *columns_to_onehotencode], axis='columns', inplace=True)

train_df.columns = train_df.columns.map(str)

x_train, x_test, y_train, y_test = train_test_split(train_df, target_df, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50, random_state=42)
rf.fit(x_train,y_train)
predictions = rf.predict(x_test)

# Evaluate

In [ ]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
mse = mean_squared_error(predictions, y_test)
mae = mean_absolute_error(predictions, y_test)
print(f'mean squared error: {mse}')
print(f'mean absolute error: {mae}')